<a href="https://colab.research.google.com/github/eswens13/deep_learning/blob/master/keras/cifar_10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR-10 Classifier/Autoencoder

This notebook is an exploratory exercise in convolutional neural networks.  I will build a classifier for the CIFAR-10 image set and play around with network architecture, hyperparameters, visualization techniques, etc. to get hands-on experience coding convolutional neural networks in TensorFlow.

I will also explore the differences between a classifier and an auto-encoder.

## Define Network Architecture

First, we have to define a network architecture.  The code in the cell below has comments explaining the architecture of each of the layers.

In [1]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D
from keras.optimizers import Adam

import numpy as np

# Build the Keras model
def get_keras_model():
  
  model = Sequential();
  num_filters = 32
  kernel_size = [3,3]
  stride_size = [1,1]
  pad_str = 'same'
  format_str = 'channels_last'
  act_str = 'relu'
  
  model.add(Conv2D(num_filters, \
                   kernel_size, \
                   strides=stride_size, \
                   padding=pad_str, \
                   data_format=format_str, \
                   use_bias=True, \
                   activation=act_str))
  
  pool_window = [2,2]
  stride_size = [2,2]
  pad_str = 'valid'
  model.add(MaxPooling2D(pool_size=pool_window, \
                         strides=stride_size, \
                         padding=pad_str, \
                         data_format=format_str))

  num_filters = 64
  pad_str = 'same'
  model.add(Conv2D(num_filters, \
                   kernel_size, \
                   strides=stride_size, \
                   padding=pad_str, \
                   data_format=format_str, \
                   use_bias=True, \
                   activation=act_str))
  
  pad_str = 'valid'
  model.add(MaxPooling2D(pool_size=pool_window, \
                         strides=stride_size, \
                         padding=pad_str, \
                         data_format=format_str))
  
  num_filters = 128
  pad_str = 'same'
  model.add(Conv2D(num_filters, \
                   kernel_size, \
                   strides=stride_size, \
                   padding=pad_str, \
                   data_format=format_str, \
                   use_bias=True, \
                   activation=act_str))
  
  pad_str = 'valid'
  model.add(MaxPooling2D(pool_size=pool_window, \
                         strides=stride_size, \
                         padding=pad_str, \
                         data_format=format_str))
  
  model.add(Flatten(data_format=format_str))
  
  model.add(Dense(512, activation=act_str, use_bias=True))
  model.add(Dense(128, activation=act_str, use_bias=True))
  
  # Output Layer
  model.add(Dense(10, activation=act_str, use_bias=True))
  
  # Compile the model.
  adam = Adam(lr=1e-4)
  model.compile(loss='categorical_crossentropy', \
                optimizer=adam, \
                metrics=['accuracy'])
  
  return model

Using TensorFlow backend.


## Bring in Data

In order to actually train the model, we need to bring in actual data.  Download the CIFAR-10 dataset and get it into a format that we can use.

In [2]:
# I'm cheating and using Keras to import the dataset without having to do a lot
# of processing myself.
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Change the labels to one-hot vectors.
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Examine what the data looks like.
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

170500096/170498071 [==============================] - 14s 0us/step
X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 10)
X_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 10)


In [3]:
import os

# Get a tool called ngrok to use as a tunnel between my local machine and the
# Google Colab server. This will allow us to use TensorBoard to visualize
# helpful metrics of the network.
#
# Tutorial:
#   https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
#
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Now the ngrok exectuable is extracted to the current directory. Check to make
# sure there is a log directory for Keras to use.
cwd = os.getcwd()
LOG_DIR = os.path.join(cwd, 'log')
print("Log Dir: {}".format(LOG_DIR))
if not os.path.exists(LOG_DIR):
  os.system('mkdir -p {}'.format(LOG_DIR))

# Run tensorboard in the background.
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# Tell ngrok (in the background) to tunnel TensorBoard port 6006 to the outside
# world.
get_ipython().system_raw('./ngrok http 6006 &')

# Get the URL that I can use to hook into TensorBoard from my local machine.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-05-25 02:55:46--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.73.94.166, 52.4.75.11, 3.214.163.243, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.73.94.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  73.2MB/s    in 0.2s    

2019-05-25 02:55:46 (73.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Log Dir: /content/log
https://5b154677.ngrok.io


In [0]:
# Create a Keras callback so that it outputs to TensorBoard rather than this
# console.
from keras.callbacks import TensorBoard

tbCallback = TensorBoard(log_dir=LOG_DIR, \
                         histogram_freq=1, \
                         write_graph=False, \
                         write_grads=False, \
                         batch_size=1000, \
                         write_images=False)

## Run Training

Run the training loop for 100 batches of images (happens fairly fast) and investigate the effectiveness of the network.

In [5]:
model = get_keras_model()
model.fit(X_train, y_train, \
          epochs=10, batch_size=1000, verbose=1, callbacks=[tbCallback], \
          validation_data=(X_test, y_test))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 6s 122us/step - loss: 2.5816 - acc: 0.1416 - val_loss: 2.2361 - val_acc: 0.1817
Epoch 2/10
50000/50000 [==============================] - 2s 44us/step - loss: 2.1588 - acc: 0.2356 - val_loss: 2.0523 - val_acc: 0.2610
Epoch 3/10
50000/50000 [==============================] - 2s 42us/step - loss: 2.1403 - acc: 0.1628 - val_loss: 2.0636 - val_acc: 0.2583
Epoch 4/10
50000/50000 [==============================] - 2s 40us/step - loss: 2.0238 - acc: 0.2926 - val_loss: 1.9930 - val_acc: 0.3183
Epoch 5/10
50000/50000 [==============================] - 2s 41us/step - loss: 1.9507 - acc: 0.3233 - val_loss: 1.9527 - val_acc: 0.2800
Epoch 6/10
50000/50000 [==============================] - 2s 43us/step - loss: 1.9402 - acc: 0.3100 - val_loss: 2.1367 - val_acc: 0.1119
Epoch 7

## Validate the Network

Run the network against the validation data set to evaluate the classification effectiveness.

In [4]:
scores = model.evaluate(X_test, y_test)
print("Accuracy: {}".format(scores))

10000/10000 [==============================] - 1s 69us/step
Accuracy: [2.7930280409812926, 0.6548]


# What Did I Learn?

I had forgotten that "epoch" does not mean the same thing as "batch".  An epoch is one pass through all of the training data.  The batch size determines how often you update the network weights.  A larger batch sizes means less batches per epochs and less updates.  With a smaller batch size, more backward passes are performed and the epochs take a much longer time.